# Portuguese Bank Project
___

# プロジェクト概要 

"Portuguese Bank Project"はポルトガル銀行の電話によるダイレクトマーケティングキャンペーンに関するプロジェクトです。
ポルトガル銀行はどのような特徴のある顧客に電話をかけたら定期預金をして貰えるかが知りたいのです。
このプロジェクトの目標は顧客リストの中から誰が定期預金をしているかを予想する事です。

このプロジェクトで用意されている顧客データの特徴量

- age: 年齢
- job: 職業　(categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
- marital: 配偶者の有無　(categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
- education: 学歴　(categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
- default: 過去にローンの支払いの遅れなどの不履行の有無 (categorical: 'no','yes','unknown')
- balance:  口座残高
- housing: 住宅ローンの有無 (categorical: 'no','yes','unknown')
- loan: 個人ローンの有無 (categorical: 'no','yes','unknown')

- contact: 連絡手段 (categorical: 'cellular','telephone')
- day: 前回このキャンペーンで連絡を取った日 (categorical: 1 ~ 31)
- month: 前回このキャンペーンで連絡を取った月 (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
- duration: 前回このキャンペーンで連絡を取った時に話をした時間(秒)

- campaign: この顧客が口座開設後にこのキャンペーンを行った回数
- pdays: 前回この顧客にこのキャンペーンで連絡を取った日から何日経過しているか (-1は連絡を取っていないと言う意味)
- previous: 過去何回このキャンペーンで連絡を取ったか
- poutcome: 過去にこのキャンペーンで連絡を取って定期預金をして貰えたことがあるかどうか(categorical: 'failure','nonexistent','success')

- y(ターゲット): 定期預金をしているかどうか(binary: 'yes','no')


In [1]:
#1
%%capture
%load_ext sql

In [2]:
#2
%%sql
postgresql:///bank

In [3]:
#3
#データセットを確認
%%sql

SELECT * FROM bank Limit  5;

 * postgresql:///bank
5 rows affected.


id,age,job,marital,education,default_,balance,housing,loan,contact,day,month,duration,aign,pdays,previous,poutcome,y
1,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
2,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
3,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
4,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
5,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
#4
#ターゲット('y')を確認
%%sql
WITH bank_y AS(
SELECT
    y,
    COUNT(*) as COUNT
FROM bank
GROUP BY y)

SELECT
     y,
    count,
    ROUND(count / (SELECT SUM(count) FROM bank_y), 2) as percentage
FROM bank_y;

 * postgresql:///bank
2 rows affected.


y,count,percentage
no,39922,0.88
yes,5289,0.12


In [130]:
#5
#ターゲット('y')と'age','balence','duration'の平均の関係を確認
%%sql

SELECT
    y,
    ROUND(AVG(age), 2) as avg_age,
    ROUND(AVG(balance), 2) as avg_balance,
    ROUND(AVG(duration), 2) as avg_duration
FROM bank
GROUP BY y;

 * postgresql:///bank
2 rows affected.


y,avg_age,avg_balance,avg_duration
no,40.84,1303.71,221.18
yes,41.67,1804.27,537.29


In [47]:
#6
#'age'を各世代にグループ分けして'y'との関係を確認する(yesとnoのカウントを確認する)
%%sql

SELECT
    bk.y,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age < 30 AND bk.y = sub_bk.y) as age_20s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 29 AND sub_bk.age < 40  AND bk.y = sub_bk.y) as age_30s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 39 AND sub_bk.age < 50  AND bk.y = sub_bk.y) as age_40s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 49 AND sub_bk.age < 60  AND bk.y = sub_bk.y) as age_50s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 59 AND bk.y = sub_bk.y) as age_over_60
FROM bank as bk
GROUP BY y;

 * postgresql:///bank
2 rows affected.


y,age_20s,age_30s,age_40s,age_50s,age_over_60
no,4345,16176,10592,7625,1184
yes,928,1913,1063,785,600


In [103]:
#7
#'age'を各世代にグループ分けして'y'との関係を確認する(yes(定期預金をしている人)の割合を確認する)
%%sql

WITH bank_age_group AS(
SELECT
    bk.y,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age < 30 AND bk.y = sub_bk.y) as age_20s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 29 AND sub_bk.age < 40  AND bk.y = sub_bk.y) as age_30s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 39 AND sub_bk.age < 50  AND bk.y = sub_bk.y) as age_40s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 49 AND sub_bk.age < 60  AND bk.y = sub_bk.y) as age_50s,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.age > 59 AND bk.y = sub_bk.y) as age_over_60
FROM bank as bk
GROUP BY y)

SELECT 
    ROUND(age_20s / (SELECT SUM(age_20s) FROM bank_age_group), 2)  as age_20s_yes_percent ,
    ROUND(age_30s / (SELECT SUM(age_30s) FROM bank_age_group), 2)  as age_30s_yes_percent ,
    ROUND(age_40s / (SELECT SUM(age_40s) FROM bank_age_group), 2)  as age_40s_yes_percent ,
    ROUND(age_50s / (SELECT SUM(age_50s) FROM bank_age_group), 2)  as age_50s_yes_percent ,
    ROUND(age_over_60 / (SELECT SUM(age_over_60) FROM bank_age_group), 2)  as age_60s_yes_percent 
FROM bank_age_group
WHERE y = 'yes';

 * postgresql:///bank
1 rows affected.


age_20s_yes_percent,age_30s_yes_percent,age_40s_yes_percent,age_50s_yes_percent,age_60s_yes_percent
0.18,0.11,0.09,0.09,0.34


In [132]:
#8
#各職業ごとのyとの関係を確認する
%%sql

SELECT job, 
              COUNT(CASE WHEN y = 'yes' THEN id END) AS y_yes,
              COUNT(CASE WHEN y = 'no' THEN id END) AS y_no,
              ROUND(AVG(CASE WHEN y = 'yes' THEN 1
                                 WHEN y = 'no' THEN 0
                                 END), 2) AS yes_percentage
FROM bank
GROUP BY job;

 * postgresql:///bank
12 rows affected.


job,y_yes,y_no,yes_percentage
retired,516,1748,0.23
housemaid,109,1131,0.09
admin.,631,4540,0.12
management,1301,8157,0.14
blue-collar,708,9024,0.07
entrepreneur,123,1364,0.08
services,369,3785,0.09
technician,840,6757,0.11
unknown,34,254,0.12
self-employed,187,1392,0.12


In [139]:
#9
#'housing'(住宅ローン)とyの関係を確認する
%%sql

SELECT
    housing,
    COUNT(*) as count,
    COUNT(CASE WHEN y = 'yes' THEN id END) AS y_yes,
    COUNT(CASE WHEN y = 'no' THEN id END) AS y_no,
    ROUND(AVG(CASE WHEN y = 'yes' THEN 1
                                 WHEN y = 'no' THEN 0
                                 END), 2) AS yes_percentage
FROM bank
GROUP BY housing;

 * postgresql:///bank
2 rows affected.


housing,count,y_yes,y_no,yes_percentage
no,20081,3354,16727,0.17
yes,25130,1935,23195,0.08


In [140]:
#10
#'loan'(個人ローン)とyの関係を確認する
%%sql

SELECT
    loan,
    COUNT(*) as count,
    COUNT(CASE WHEN y = 'yes' THEN id END) AS y_yes,
    COUNT(CASE WHEN y = 'no' THEN id END) AS y_no,
    ROUND(AVG(CASE WHEN y = 'yes' THEN 1
                                 WHEN y = 'no' THEN 0
                                 END), 2) AS yes_percentage
FROM bank
GROUP BY loan;

 * postgresql:///bank
2 rows affected.


loan,count,y_yes,y_no,yes_percentage
no,37967,4805,33162,0.13
yes,7244,484,6760,0.07


In [141]:
#11
#'housing'(住宅ローン)と'loan'(個人ローン)を掛け合わせた特徴量とyの関係を確認する(yesとnoのカウントを確認する)
%%sql

SELECT
    bk.y,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'no' AND sub_bk.loan = 'no' AND bk.y = sub_bk.y) as housing_no_loan_no,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'no' AND sub_bk.loan = 'yes' AND bk.y = sub_bk.y) as housing_no_loan_yes,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'yes' AND sub_bk.loan = 'no' AND bk.y = sub_bk.y) as housing_yes_loan_no,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'yes' AND sub_bk.loan = 'no'  AND bk.y = sub_bk.y) as housing_yes_loan_yes
FROM bank as bk
GROUP BY y;

 * postgresql:///bank
2 rows affected.


y,housing_no_loan_no,housing_no_loan_yes,housing_yes_loan_no,housing_yes_loan_yes
no,14069,2658,19093,19093
yes,3135,219,1670,1670


In [159]:
#12
#'housing'(住宅ローン)と'loan'(個人ローン)を掛け合わせた特徴量とyの関係を確認する(yes(定期預金をしている人)の割合を確認する)
%%sql

WITH bank_loan_group AS(
    SELECT
    bk.y,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'no' AND sub_bk.loan = 'no' AND bk.y = sub_bk.y) as housing_no_loan_no,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'no' AND sub_bk.loan = 'yes' AND bk.y = sub_bk.y) as housing_no_loan_yes,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'yes' AND sub_bk.loan = 'no' AND bk.y = sub_bk.y) as housing_yes_loan_no,
    (SELECT COUNT(*) FROM bank as sub_bk
      WHERE sub_bk.housing = 'yes' AND sub_bk.loan = 'yes'  AND bk.y = sub_bk.y) as housing_yes_loan_yes
    FROM bank as bk
    GROUP BY y
)

SELECT 
    ROUND(housing_no_loan_no / (SELECT SUM(housing_no_loan_no) FROM bank_loan_group), 2)  as housing_no_loan_no_yes_percent,
    ROUND(housing_no_loan_yes / (SELECT SUM(housing_no_loan_yes) FROM bank_loan_group), 2)  as housing_no_loan_yes_yes_percent,
    ROUND(housing_yes_loan_no / (SELECT SUM(housing_yes_loan_no) FROM bank_loan_group), 2)  as housing_yes_loan_no_yes_percent,
    ROUND(housing_yes_loan_yes / (SELECT SUM(housing_yes_loan_yes) FROM bank_loan_group), 2)  as housing_yes_loan_yes_yes_percent
FROM bank_loan_group
WHERE y = 'yes';

 * postgresql:///bank
1 rows affected.


housing_no_loan_no_yes_percent,housing_no_loan_yes_yes_percent,housing_yes_loan_no_yes_percent,housing_yes_loan_yes_yes_percent
0.18,0.08,0.08,0.06
